In [ ]:
pip install scikit-learn
pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 95.3 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing import image
import cv2

In [ ]:
# Chargement des images
def load_images_from_folder(folder, label, target_size=(224, 224)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, target_size)
            images.append(img)
            labels.append(label)
    return images, labels

# Charger les données
def load_data():
    # Chemins vers vos sous-dossiers
    pop_folder = "data/Pop"
    jazz_folder = "data/Jazz"
    death_metal_folder = "data/Death_Metal"

    pop_images, pop_labels = load_images_from_folder(pop_folder, 0)  # 0 pour Pop
    jazz_images, jazz_labels = load_images_from_folder(jazz_folder, 1)  # 1 pour Jazz
    death_metal_images, death_metal_labels = load_images_from_folder(death_metal_folder, 2)  # 2 pour Death Metal

    # Combiner les données
    images = np.array(pop_images + jazz_images + death_metal_images)
    labels = np.array(pop_labels + jazz_labels + death_metal_labels)

    return images, labels

# Prétraitement des images : normalisation et vectorisation
def preprocess_images(images):
    # Normalisation : Diviser par 255 pour que les pixels soient dans [0, 1]
    images = images.astype('float32') / 255.0

    # Aplatir les images en vecteurs (224 x 224 x 3 -> 150528)
    images = images.reshape(images.shape[0], -1)
    
    # Application de PCA pour réduire la dimensionnalité (environ 500 dimensions pour un bon compromis)
    pca = PCA(n_components=500)
    images = pca.fit_transform(images)
    
    return images


In [7]:
# Charger les données
images, labels = load_data()

In [11]:
# Print the number of genre in all the labels

unique, counts = np.unique(labels, return_counts=True)
label_counts = dict(zip(unique, counts))
print(label_counts)

{np.int64(0): np.int64(948), np.int64(1): np.int64(943), np.int64(2): np.int64(889)}


In [12]:
# Diviser les données en training et test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [13]:
# Prétraiter les images
X_train = preprocess_images(X_train)
X_test = preprocess_images(X_test)

# Standardisation (important pour SVM et KNN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
def evaluate_models(X_train, X_test, y_train, y_test):
    models = {
        "KNN": KNeighborsClassifier(n_neighbors=60),
        "SVM": SVC(),
        "Logistic Regression": LogisticRegression(max_iter=1000)
    }

    for name, model in models.items():
        # Validation croisée
        print(f"Validation croisée pour {name} :")
        scores = cross_val_score(model, X_train, y_train, cv=5)
        print(f"Cross-validation accuracy: {scores.mean():.4f} ± {scores.std():.4f}")
        
        # Entraînement et évaluation du modèle
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        print(f"{name} Accuracy: {accuracy:.4f}")
        print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

In [18]:
# Evaluer les modèles
evaluate_models(X_train, X_test, y_train, y_test)

Validation croisée pour KNN :
Cross-validation accuracy: 0.3898 ± 0.0357
KNN Accuracy: 0.3417
Classification Report:
              precision    recall  f1-score   support

           0       0.34      0.82      0.48       187
           1       0.00      0.00      0.00       192
           2       0.35      0.21      0.26       177

    accuracy                           0.34       556
   macro avg       0.23      0.34      0.25       556
weighted avg       0.23      0.34      0.24       556

Validation croisée pour SVM :


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Cross-validation accuracy: 0.4856 ± 0.0171
SVM Accuracy: 0.3993
Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.35      0.40       187
           1       0.36      0.54      0.43       192
           2       0.39      0.31      0.34       177

    accuracy                           0.40       556
   macro avg       0.41      0.40      0.39       556
weighted avg       0.41      0.40      0.39       556

Validation croisée pour Logistic Regression :
Cross-validation accuracy: 0.4429 ± 0.0161
Logistic Regression Accuracy: 0.3669
Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.38      0.37       187
           1       0.37      0.35      0.36       192
           2       0.36      0.37      0.37       177

    accuracy                           0.37       556
   macro avg       0.37      0.37      0.37       556
weighted avg       0.37      0.37      0.37       556



CNN simple que je fais moi meme :

CNN deja entrainé (et puissant)

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Charger MobileNetV2 préentraîné
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Ajouter une tête de classification
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Réduction de dimensions
x = Dense(128, activation='relu')(x)  # Couche intermédiaire dense
predictions = Dense(3, activation='softmax')(x)  # Couche finale pour 3 classes

# Créer le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Geler les poids du modèle de base
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


I0000 00:00:1735840731.707069    7897 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13764 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Générer des données augmentées
datagen = ImageDataGenerator(
    rescale=1.0 / 255,         # Normaliser les pixels entre 0 et 1
    rotation_range=15,         # Rotation aléatoire
    width_shift_range=0.1,     # Décalage horizontal
    height_shift_range=0.1,    # Décalage vertical
    horizontal_flip=True       # Renversement horizontal
)

# Préparer les données pour l'entraînement et la validation
train_generator = datagen.flow(X_train, y_train, batch_size=32)
validation_generator = datagen.flow(X_test, y_test, batch_size=32)

ValueError: Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape (2224, 500)

In [ ]:
# Entraîner le modèle
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    steps_per_epoch=len(X_train) // 32,
    validation_steps=len(X_test) // 32
)


In [ ]:
# Évaluer le modèle
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Afficher les courbes d'entraînement
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()
